In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


display(x_train.head())
display(y_train.head())

x_df = pd.merge(x_train, y_train, on='ID')

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [2]:
print(x_df.shape,'\n')
print(x_df.head(3), '\n')
print(x_df.info(),'\n')
print(x_df.describe,'\n')
print(x_df.isnull().sum(),'\n')

(6598, 12) 

     ID Warehouse_block Mode_of_Shipment Customer_care_calls  Customer_rating  \
0  6045               A           Flight                   4                3   
1    44               F             Ship                   3                1   
2  7940               F             Road                   4                1   

   Cost_of_the_Product  Prior_purchases Product_importance Gender  \
0                  266                5               high      F   
1                  174                2                low      M   
2                  154               10               high      M   

   Discount_offered  Weight_in_gms  Reached.on.Time_Y.N  
0                 5           1590                    0  
1                44           1556                    1  
2                10           5674                    1   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dt

In [3]:
x = x_df.drop(['ID','Reached.on.Time_Y.N'], axis=1)
test = x_test.drop(['ID'],axis=1)
y = x_df['Reached.on.Time_Y.N']

In [4]:
obj_col = x_df.select_dtypes(include='object').columns
num_col = x_df.select_dtypes(exclude='object').columns

print(obj_col, '\n')
print(num_col)

Index(['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Product_importance', 'Gender'],
      dtype='object') 

Index(['ID', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Discount_offered', 'Weight_in_gms', 'Reached.on.Time_Y.N'],
      dtype='object')


In [5]:
for col in obj_col:
  print(col,'\n')
  print(x[col].unique(), '\n')

Warehouse_block 

['A' 'F' 'C' 'B' 'D'] 

Mode_of_Shipment 

['Flight' 'Ship' 'Road'] 

Customer_care_calls 

['4' '3' '5' '6' '2' '$7'] 

Product_importance 

['high' 'low' 'medium'] 

Gender 

['F' 'M'] 



In [6]:
le_col = ['Warehouse_block', 'Customer_care_calls']
dum_col = ['Mode_of_Shipment','Product_importance','Gender']
mms_col = ['Customer_rating', 'Cost_of_the_Product', 'Prior_purchases','Discount_offered', 'Weight_in_gms']

In [7]:
from sklearn.preprocessing import MinMaxScaler
for col in mms_col:
  mms = MinMaxScaler()
  x[col] = mms.fit_transform(x[[col]])
  test[col] = mms.transform(test[[col]])

In [8]:
from sklearn.preprocessing import LabelEncoder

for col in le_col:
  le = LabelEncoder()
  x[col] = le.fit_transform(x[col])
  test[col] = le.transform(test[col])

In [9]:
x = pd.get_dummies(x, columns=dum_col, dtype=int)
test = pd.get_dummies(test, columns=dum_col, dtype=int)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, stratify=y,random_state=1234)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=1234)

params = {'n_estimators':[5,10,15,20], 'max_depth':[5,10,15,20,25,30]}
gscv = GridSearchCV(estimator=rf, param_grid=params, cv=5)
gscv.fit(x_train,y_train)
print(gscv.best_params_)

{'max_depth': 5, 'n_estimators': 10}


In [ ]:
model = RandomForestClassifier(max_depth=5, n_estimators=10, random_state=1234)
model.fit(x_train,y_train)
model_res = model.predict(x_val)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score

print(accuracy_score(model_res, y_val))
print(confusion_matrix(model_res, y_val))
print(f1_score(y_val,model_res, average='macro'))
print(roc_auc_score(model_res, y_val))

0.6787878787878788
[[620 457]
 [179 724]]
0.6778992342919349
0.6887225188708639


In [ ]:
pred = model.predict(test)

result = pd.DataFrame({'ID':x_test['ID'], 'pred':pred})
result.to_csv('수험번호.csv', index=False)
print(result)

         ID  pred
0      6811     1
1      4320     0
2      5732     0
3      7429     0
4      2191     1
...     ...   ...
4396   2610     1
4397   3406     0
4398  10395     0
4399   3646     0
4400    573     1

[4401 rows x 2 columns]
